# first attempt - simple reverse prediction

In [3]:
import numpy as np
import pandas as pd
from functions import *
from read_from_file_df import *
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [4]:
gen = 2
name_df = f'{PATH_DF}\\{SIZE}size_{AMOUNT_BOARDS}boards_{gen}gen_reverse'
reverse_df = pd.read_pickle(f'{name_df}.pkl')

In [5]:
new_columns = [f'Col_{i}' for i in range(1, gen*SIZE*SIZE + 1)]
reverse_df_sort = reverse_df.sort_values(by = new_columns).reset_index(drop=True)
for i in reverse_df_sort.columns:
    reverse_df_sort[i] = reverse_df_sort[i].astype(int)

In [6]:
print("reverse df:", len(reverse_df))
print("reverse df sort:",len(reverse_df_sort))

reverse df: 2870515
reverse df sort: 2870515


In [7]:
# Step 1: Prepare Data
amount_features = len(reverse_df_sort.columns) - SIZE*SIZE #the previous boards
features = reverse_df_sort.iloc[:, :amount_features]
name_col = 'Col_' + str(amount_features + 1)  # Target: the first pixel in the board
target = reverse_df_sort[name_col]

# Step 2: Split Data
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=365)

print("len x train: ", len(X_train))
print("len x test: ",len(X_test))
print("len y train: ",len(y_train))
print("len y test: ",len(y_test))

dt = tree.DecisionTreeClassifier(random_state=42)
dt = dt.fit(X_train, y_train)

y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)

train_test_full_error = pd.concat([measure_error(y_train, y_train_pred, 'train'),
                                measure_error(y_test, y_test_pred, 'test')],
                                axis=1)

print(dt.tree_.node_count, dt.tree_.max_depth)
train_test_full_error

len x train:  2583463
len x test:  287052
len y train:  2583463
len y test:  287052
657685 100


,train,test
accuracy,0.999279,0.788031
precision,0.999562,0.492274
recall,0.996991,0.528578
f1,0.998275,0.509781


In [8]:
reverse_df_sort.iloc[21540].values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [9]:
features.iloc[21540].values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [10]:
target.iloc[21540]

np.int64(0)

In [11]:
depth_to_check = [4,5,6,7,8,9,10,15,20,30,50,100]

for i in depth_to_check:
    dt, node_count, max_depth, train_test_full_error  = dec_tree_df(X_train,y_train, X_test, y_test, i, 42)
    
    name_file = f"{PATH_MODELS}\\reverse_model1\\decision_tree_reverse_df_model_{str(i)}.pkl"
    with open(name_file, 'wb') as file:
        pickle.dump(dt, file)
    
    print(node_count, max_depth)
    print(train_test_full_error)

31 4
              train      test
accuracy   0.816114  0.815681
precision  0.571896  0.568416
recall     0.480495  0.481947
f1         0.522226  0.521623
63 5
              train      test
accuracy   0.821633  0.821430
precision  0.639757  0.635279
recall     0.336909  0.337143
f1         0.441379  0.440508
127 6
              train      test
accuracy   0.823187  0.822893
precision  0.627837  0.623669
recall     0.379707  0.379747
f1         0.473218  0.472060
255 7
              train      test
accuracy   0.826083  0.825910
precision  0.651121  0.646901
recall     0.362943  0.363457
f1         0.466085  0.465421
511 8
              train      test
accuracy   0.827742  0.827073
precision  0.645801  0.640596
recall     0.390674  0.388769
f1         0.486838  0.483879
1023 9
              train      test
accuracy   0.830808  0.830198
precision  0.654431  0.648388
recall     0.404833  0.405577
f1         0.500225  0.499013
2047 10
              train      test
accuracy   0.834221  0.8330

In [12]:
amount_features = len(reverse_df_sort.columns) - SIZE*SIZE #the previous boards
features = reverse_df_sort.iloc[:, :amount_features]
for i in range(SIZE*SIZE): # to any pixel in the expected board
    name_col = 'Col_' + str(i+amount_features + 1)
    target = reverse_df_sort[name_col]
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=613)
    dt, node_count, max_depth, train_test_full_error  = dec_tree_df(X_train,y_train, X_test, y_test, 10, 42)
    name_file = f"{PATH_MODELS}\\reverse1\\dt_r1_pixel_{str(i+1)}.pkl"
    with open(name_file, 'wb') as file:
        pickle.dump(dt, file)
    print(i)
    print(node_count, max_depth)
    print(train_test_full_error)

0
2047 10
              train      test
accuracy   0.834448  0.833657
precision  0.664107  0.660212
recall     0.421592  0.418736
f1         0.515764  0.512452
1
2047 10
              train      test
accuracy   0.833600  0.833243
precision  0.660733  0.657448
recall     0.425073  0.421992
f1         0.517330  0.514040
2
2047 10
              train      test
accuracy   0.833965  0.833002
precision  0.663279  0.656535
recall     0.420062  0.417814
f1         0.514368  0.510652
3
2047 10
              train      test
accuracy   0.834079  0.832630
precision  0.660949  0.659029
recall     0.423631  0.422293
f1         0.516326  0.514746
4
2047 10
              train      test
accuracy   0.833907  0.831612
precision  0.666744  0.661287
recall     0.414291  0.409681
f1         0.511040  0.505929
5
2047 10
              train      test
accuracy   0.833864  0.832651
precision  0.666034  0.664670
recall     0.412882  0.411863
f1         0.509758  0.508583
6
2047 10
              train      test
